#**COMP1804 - Task 2**

#**Imports**

In [67]:
!pip install gdown
import gdown
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.dummy import DummyClassifier
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Data Loading**

In [ ]:
data = pd.read_csv("Dataset.csv")

Downloading...
From: https://drive.google.com/uc?id=1N3Sw39P76RaN3ftEWTwvIK4CBBqe04Cr
To: /content/comp1804_coursework_dataset_24-25-copy.csv
100%|██████████| 6.81M/6.81M [00:00<00:00, 38.5MB/s]


In [69]:
data

,petition_ID,has_entity,relevant_department,deviation_across_regions,petition_status,petition_topic,petition_importance,petition_text
0,607158,EVENT:NO_DATE:NO_PERSON:NO_,Department for Education,0.000137,unsuccessful,education,not_important,Ban school uniforms in English schools. I want...
1,590629,EVENT:NO_DATE:YES_PERSON:NO_,Department of Health and Social Care,0.002133,unsuccessful,uk government and devolution,important,Lock NHS pay rises to increases in pay for MPs...
2,652724,EVENT:NO_DATE:NO_PERSON:NO_,Department of Health and Social Care,0.000272,unsuccessful,health and social care,important,Require dose counters on SABA metered dose inh...
3,629281,EVENT:NO_DATE:NO_PERSON:NO_,HM Treasury,0.000083,unsuccessful,"economy, labour and welfare",not_important,Remove the 20% VAT on private dementia day car...
4,301747,EVENT:NO_DATE:NO_PERSON:NO_,NaN,NaN,rejected,"culture, sport and media",not_important,Make bank holidays a holiday. Make bank holida...
...,...,...,...,...,...,...,...,...
8893,569251,EVENT:NO_DATE:NO_PERSON:YES_,Department of Health and Social Care,0.000109,unsuccessful,health and social care,NaN,Fund free meals for NHS frontline workers whil...
8894,616956,EVENT:NO_DATE:YES_PERSON:YES_,HM Treasury,0.000051,unsuccessful,environment and animal welfare,NaN,Cap profit fuel forecourt operators can make o...
8895,302075,data missing,NaN,NaN,rejected,"culture, sport and media",NaN,Coronavirus. Re-introduce free TV license for ...
8896,326309,EVENT:NO_DATE:NO_PERSON:NO_,"Department for Digital, Culture, Media and Sport",0.000077,unsuccessful,"culture, sport and media",NaN,Restrict the share size of individuals and com...


#**Data Cleaning**

In [70]:
print(data.isnull().sum())

petition_ID                    0
has_entity                     0
relevant_department         1799
deviation_across_regions    1577
petition_status                0
petition_topic                31
petition_importance         8798
petition_text                  2
dtype: int64


In [71]:
data['deviation_across_regions'] = data['deviation_across_regions'].fillna(data['deviation_across_regions'].median())
data['petition_topic'] = data['petition_topic'].fillna('unknown')

In [72]:
print(data.isnull().sum())

petition_ID                    0
has_entity                     0
relevant_department         1799
deviation_across_regions       0
petition_status                0
petition_topic                 0
petition_importance         8798
petition_text                  2
dtype: int64


#**Text Preprocessing**

In [73]:
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        return ' '.join(tokens)
    else:
        return ""

data['petition_text'] = data['petition_text'].apply(preprocess_text)

In [74]:
data['petition_text']

,petition_text
0,ban school uniform english school want governm...
1,lock nh pay rise increase pay mp backdated 201...
2,require dose counter saba metered dose inhaler...
3,remove 20 vat private dementia day care centre...
4,make bank holiday holiday make bank holiday ho...
...,...
8893,fund free meal nh frontline worker shift provi...
8894,cap profit fuel forecourt operator make petrol...
8895,coronavirus reintroduce free tv license senior...
8896,restrict share size individual company medium ...


#**Feature Encoding**

In [75]:
labeled_data = data[data['petition_importance'].notna()]
unlabeled_data = data[data['petition_importance'].isna()]


print(f"Labeled data size: {len(labeled_data)} rows")
print(f"Unlabeled data size: {len(unlabeled_data)} rows")

Labeled data size: 100 rows
Unlabeled data size: 8798 rows


In [76]:
labeled_data['petition_importance'].value_counts()

,count
petition_importance,
important,59
not_important,41


In [77]:
labeled_data.iloc[0]

,0
petition_ID,607158
has_entity,EVENT:NO_DATE:NO_PERSON:NO_
relevant_department,Department for Education
deviation_across_regions,0.000137
petition_status,unsuccessful
petition_topic,education
petition_importance,not_important
petition_text,ban school uniform english school want governm...


In [78]:
labeled_data.iloc[2]

,2
petition_ID,652724
has_entity,EVENT:NO_DATE:NO_PERSON:NO_
relevant_department,Department of Health and Social Care
deviation_across_regions,0.000272
petition_status,unsuccessful
petition_topic,health and social care
petition_importance,important
petition_text,require dose counter saba metered dose inhaler...


In [79]:
le = LabelEncoder()
y_labeled = le.fit_transform(labeled_data['petition_importance'])


In [80]:
y_labeled

array([1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1])

In [81]:
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
text_features_all = tfidf.fit_transform(data['petition_text'])
has_entity_dummies_all = pd.get_dummies(data['has_entity'], prefix='entity')
topic_dummies_all = pd.get_dummies(data['petition_topic'], prefix='topic')
status_dummies_all = pd.get_dummies(data['petition_status'], prefix='status')
deviation_all = data['deviation_across_regions'].values.reshape(-1, 1)

In [82]:
has_entity_dummies_all


,entity_EVENT:NO_DATE:NO_PERSON:NO_,entity_EVENT:NO_DATE:NO_PERSON:YES_,entity_EVENT:NO_DATE:YES_PERSON:NO_,entity_EVENT:NO_DATE:YES_PERSON:YES_,entity_EVENT:YES_DATE:NO_PERSON:NO_,entity_EVENT:YES_DATE:NO_PERSON:YES_,entity_EVENT:YES_DATE:YES_PERSON:NO_,entity_EVENT:YES_DATE:YES_PERSON:YES_,entity_data missing
0,True,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
8893,False,True,False,False,False,False,False,False,False
8894,False,False,False,True,False,False,False,False,False
8895,False,False,False,False,False,False,False,False,True
8896,True,False,False,False,False,False,False,False,False


In [83]:
topic_dummies_all

,"topic_Culture, Sport And Media","topic_Economy, Labour And Welfare",topic_Education,topic_Environment And Animal Welfare,topic_Health And Social Care,topic_London,topic_Uk Government And Devolution,"topic_culture, sport and media","topic_economy, labour and welfare",topic_education,topic_environment and animal welfare,topic_health and social care,topic_london,topic_uk government and devolution,topic_unknown
0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8893,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
8894,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
8895,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
8896,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


In [84]:
deviation_all

array([[1.371460e-04],
       [2.133273e-03],
       [2.721320e-04],
       ...,
       [1.308150e-04],
       [7.700000e-05],
       [4.291880e-04]])

#**Feature combining and data splitting**

In [85]:
X_all = hstack([text_features_all, has_entity_dummies_all, topic_dummies_all, status_dummies_all, deviation_all])

In [86]:
X_all = X_all.tocsr()
X_labeled = X_all[:len(labeled_data)]


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42, stratify=y_labeled)


#**Final Model Training with GridSeachCV**

In [88]:
base_model = LogisticRegression(class_weight='balanced', max_iter=5000, random_state=42)
param_grid = [
    {'solver': ['liblinear'],
     'penalty': ['l1', 'l2'],
     'C': [0.1]
     },

    {'solver': ['lbfgs'],
     'penalty': ['l2'],
     'C': [0.1]
     }
]
grid_search = GridSearchCV(base_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

model = grid_search.best_estimator_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [89]:
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

Best Hyperparameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best Cross-Validation Score: 0.6875


In [90]:
y_pred_test = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy on Labeled Data:", test_accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred_test))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))

Test Accuracy on Labeled Data: 0.9
Classification Report:
               precision    recall  f1-score   support

           0       0.86      1.00      0.92        12
           1       1.00      0.75      0.86         8

    accuracy                           0.90        20
   macro avg       0.93      0.88      0.89        20
weighted avg       0.91      0.90      0.90        20

Confusion Matrix:
 [[12  0]
 [ 2  6]]


In [91]:
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

[[12  0]
 [ 2  6]]


In [92]:
y_pred_all = model.predict(X_all)
data['predicted_petition_importance'] = le.inverse_transform(y_pred_all)

In [93]:
output_file = os.path.join(os.getcwd(), 'predictions.csv')
data.to_csv(output_file, index=False)
print(f"Full dataset with predictions exported to: {output_file}")

if os.path.exists(output_file):
    print("File successfully created!")
else:
    print("Error: File was not created.")

Full dataset with predictions exported to: /content/predictions.csv
File successfully created!


In [94]:
test_accuracy = model.score(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}" )

Test Accuracy: 0.90


In [95]:
prd = pd.read_csv('predictions.csv')

In [96]:
data['petition_importance'].value_counts()


,count
petition_importance,
important,59
not_important,41


In [97]:
prd['predicted_petition_importance'].value_counts()

,count
predicted_petition_importance,
important,5711
not_important,3187


In [98]:
print(prd.isnull().sum())

petition_ID                         0
has_entity                          0
relevant_department              1799
deviation_across_regions            0
petition_status                     0
petition_topic                      0
petition_importance              8798
petition_text                       2
predicted_petition_importance       0
dtype: int64


In [99]:
prd.columns

Index(['petition_ID', 'has_entity', 'relevant_department',
       'deviation_across_regions', 'petition_status', 'petition_topic',
       'petition_importance', 'petition_text',
       'predicted_petition_importance'],
      dtype='object')